In [1]:
!pip3 install --upgrade transformers optimum
!pip3 install --upgrade auto-gptq
!pip3 install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [10]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_CENTER, TA_RIGHT
from reportlab.platypus import PageTemplate, Frame

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [4]:
example = '''[INST]Here is an example of medical report/findings from a medical image generated from keywords: keywords: {
      "fracture_type": {
        "category": "long bone",
        "sub_category": "tebia",
        "fracture_subtype": "extra articular",
        "specific_subtype": "spiral"
      }
      report : "Clinical History: The patient presented with localized pain and tenderness in the left lower extremity following trauma.Findings: Imaging of the left lower extremity was performed. The study reveals a fracture involving the proximal aspect of the tibia, classified as an extra-articular fracture. The fracture pattern is characterized as a spiral fracture. These findings are consistent with a spiral extra-articular fracture of the proximal tibia.Impressions: Further clinical correlation and management, including potential orthopedic intervention, are recommended. Consultation with an orthopedic specialist for further evaluation and management planning is advised. Consideration for additional imaging studies, such as CT or MRI, if warranted for further characterization of the fracture pattern and assessment of soft tissue involvement is also recommended. Initiation of appropriate pain management and immobilization measures as indicated is essential.Conclusion: The imaging findings demonstrate a spiral extra-articular fracture of the proximal tibia. Clinical correlation and orthopedic consultation are advised for further management.[/INST]"
      [INST] Similarly,based on the given example, generate a radiological medical report for the following keywords. [/INST]'''

In [5]:
def get_fracture_severity(fracture_subtype, age):

    severity_map = {
        "humorous": "Moderate",
        "tebia": "Moderate",
        "femur": "Severe",
        "radiusalma": "Moderate",
        "febula": "Moderate",
        "knee": "Moderate",
        "anabolic different ways": "Moderate",
        "spiral": "Mild",
        "greenstick": "Mild",
        "comminuted": "Severe",
        "segmented": "Severe",
        "transverse": "Mild",
        "oblique": "Mild",
        "rest": "Mild",
        "avulsion": "Moderate",
        "impacted": "Moderate"
    }
    if fracture_subtype in severity_map:
    # Adjust severity based on age
      if age > 80:
          if severity_map[fracture_subtype] == "Mild":
              return "High"
          elif severity_map[fracture_subtype] == "Moderate":
              return "High"
          else:
              return severity_map[fracture_subtype]
      elif age > 70:
          if severity_map[fracture_subtype] == "Mild":
              return "Moderate"
          elif severity_map[fracture_subtype] == "Moderate":
              return "High"
          else:
              return severity_map[fracture_subtype]
      elif age > 50:
          if severity_map[fracture_subtype] == "Mild":
              return "Moderate"
          else:
              return severity_map[fracture_subtype]
      else:
          return severity_map[fracture_subtype]
    else:
      return "Unknown"

In [6]:
keywords = {
    "fracture_type": {
        "category": "long bone",
        "sub_category": "femur",
        "fracture_subtype": "extra articular",
        "specific_subtype": "comminuted"
    }
}


In [7]:
patient_info_keywords = {
    "patient_name": "Moe Lester",
    "gender": "male",
    "age": "56",
    "consulting_doctor": "Dr. Niggesh Joshi"
}


In [8]:
prompt = f"<s>[INST] {example} : {keywords} [/INST]"
# Inference
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.6, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
input_length = input_ids.size(1)
generated_tokens = output[:, input_length:]
generated =  tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
print(generated)

fracture_subtype = keywords['fracture_type']['specific_subtype']
age = int(patient_info_keywords['age'])
severity = get_fracture_severity(fracture_subtype, age)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clinical History: The patient presented to the emergency department with complaints of severe pain and deformity in the right lower extremity following a traumatic event.

Findings: Radiographic evaluation of the right lower extremity was performed. The images reveal a fracture involving the mid-shaft of the femur, classified as an extra-articular fracture. The fracture pattern is complex and comminuted, indicating multiple fragments and disruptions in the bone. These findings are consistent with a comminuted extra-articular fracture of the mid-shaft femur.

Impressions: Given the severity of the fracture, further clinical correlation and management, including potential orthopedic intervention, is recommended. Consultation with an orthopedic specialist for further evaluation and management planning is advised. Consideration for additional imaging studies, such as CT or MRI, to assess the degree of soft tissue involvement, potential vascular injury, and the alignment and displacement of

In [12]:
pdf_filename = "medical_report.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
styles = getSampleStyleSheet()

logo_path = "logo.png"
logo = Image(logo_path, width=100, height=100)
logo.hAlign = "RIGHT"

patient_info = [
    f"Name: {patient_info_keywords['patient_name']}",
    f"Gender: {patient_info_keywords['gender']}",
    f"Age: {patient_info_keywords['age']}",
    f"Consulting Doctor: {patient_info_keywords['consulting_doctor']}"
]


normal_style = styles["Normal"]
normal_style.alignment = 4

medical_findings = [para.strip() for para in generated.split('\n\n') if para.strip()]

pdf_filename = "medical_report.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
styles = getSampleStyleSheet()
logo_path = "logo.png"
logo = Image(logo_path, width=100, height=100)
logo.hAlign = "RIGHT"

patient_info = [
    f"Name: {patient_info_keywords['patient_name']}",
    f"Gender: {patient_info_keywords['gender']}",
    f"Age: {patient_info_keywords['age']}",
    f"Consulting Doctor: {patient_info_keywords['consulting_doctor']}"
]


report_paragraphs = [
    Paragraph("Patient Information:", styles['Heading1']),
    *[Paragraph(info, styles['Normal']) for info in patient_info],
    Spacer(1, 12),
    Paragraph("Medical Findings:", styles['Heading1']),
    *[Paragraph(para,  normal_style) for para in medical_findings],
    Spacer(1, 12),
    Paragraph("Severity:", styles['Heading1']),
    Paragraph(severity, styles['Normal']),
    Spacer(1, 12),
    Spacer(1, 12),
    Spacer(1, 12),
    Spacer(1, 12)
]

disclaimer = (
    "\n\n*Please note that this report is provided as a preliminary analysis based on the information provided and available data. "
    "We strongly advise consulting with a qualified healthcare professional promptly for further evaluation and confirmation."
)

def add_footer(canvas, doc, disclaimer):
    canvas.saveState()
    footer = Paragraph(disclaimer, styles["Italic"])
    w, h = footer.wrap(doc.width, doc.bottomMargin)
    footer.drawOn(canvas, doc.leftMargin, h)
    canvas.restoreState()

doc.build([logo] + report_paragraphs, onFirstPage=lambda canvas, doc: add_footer(canvas, doc, disclaimer), onLaterPages=lambda canvas, doc: add_footer(canvas, doc, disclaimer))


print(f"PDF report generated: {pdf_filename}")


PDF report generated: medical_report.pdf
